In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
import numpy as np
import qokit.fur as qk_fur

In [14]:
number_of_qubits = 5
hamiltonian_seed = 42
numpy_rng = np.random.default_rng()
#possibly modify
qubit_pairs = [(i,j) for i in range(number_of_qubits) for j in range(i+1, number_of_qubits)]
random_coefficients = numpy_rng.uniform(-1, 
                                        1, 
                                        size=len(qubit_pairs)).tolist()
#this is qokit's format
random_hamiltonian = [(coeff, pair) for pair, coeff in zip(qubit_pairs, random_coefficients)]
random_hamiltonian

[(-0.3505333786731821, (0, 1)),
 (-0.7021298575369499, (0, 2)),
 (-0.7504512579383398, (0, 3)),
 (0.2755093043658916, (0, 4)),
 (-0.7846357104764359, (1, 2)),
 (-0.7140149866746479, (1, 3)),
 (0.012179513765296246, (1, 4)),
 (0.3067512235551355, (2, 3)),
 (-0.8345527103728894, (2, 4)),
 (0.3064179720905926, (3, 4))]

In [16]:

qokit_backend = 'auto'
simclass = qk_fur.choose_simulator(name=qokit_backend)

          
simulator_qokit = simclass(number_of_qubits,
                       terms=random_hamiltonian)
#this computes all eigenvalues
spectrum = simulator_qokit.get_cost_diagonal()

In [17]:
spectrum

array([-3.23545989, -0.18024951,  0.43854924,  2.0916261 ,  0.79367422,
        1.04036517,  1.3291405 ,  0.17369794, -1.53286579, -1.47946044,
       -0.71491661, -2.06364478,  3.72327321,  0.96815913,  1.40267955,
       -2.75456805, -2.75456805,  1.40267955,  0.96815913,  3.72327321,
       -2.06364478, -0.71491661, -1.47946044, -1.53286579,  0.17369794,
        1.3291405 ,  1.04036517,  0.79367422,  2.0916261 ,  0.43854924,
       -0.18024951, -3.23545989])

In [32]:
gammas_j = np.array([0.2*np.pi, 0.3*np.pi])
betas_j = np.array([0.1*np.pi, 0.2*np.pi])
#multiply by 2 to get exp(-1j*gamma*H) operator
results_qokit = simulator_qokit.simulate_qaoa(gammas_j * 2, betas_j)
#get ket
statevector_ideal = simulator_qokit.get_statevector(results_qokit).reshape(-1, 1)
#get exp value
energy_expected_value = simulator_qokit.get_expectation(results_qokit)

np.float64(0.5103305874247175)